# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [12]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [13]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Dataset/Dataset_clustering.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df = pd.read_csv(file_path)

In [15]:
df.head()

,TransactionAmount,TransactionDuration,CustomerAge,AccountBalance,Cluster,TransactionType,Channel
0,14.09,81.0,70.0,5112.21,4,Debit,ATM
1,376.24,141.0,68.0,13758.91,4,Debit,ATM
2,126.29,56.0,19.0,1122.35,1,Debit,Online
3,184.50,25.0,26.0,8569.06,1,Debit,Online
4,13.45,198.0,26.0,7429.40,3,Credit,Online


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [16]:
le = LabelEncoder()
df[['TransactionType', 'Channel']] = df[['TransactionType', 'Channel']].apply(le.fit_transform)

In [17]:
X = df.drop(columns=['Cluster'])
y = df['Cluster'] - df['Cluster'].min()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [18]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)



In [19]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)


y_pred_dt = dt_model.predict(X_test)



Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [20]:
# Evaluasi Model
print("=== Random Forest ===")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

=== Random Forest ===
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       113
           1       1.00      1.00      1.00       124
           2       1.00      1.00      1.00       114
           3       1.00      1.00      1.00       152

    accuracy                           1.00       503
   macro avg       1.00      1.00      1.00       503
weighted avg       1.00      1.00      1.00       503



In [21]:
# Evaluasi Model
print("=== Decision Tree ===")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Classification Report:\n", classification_report(y_test, y_pred_dt))

=== Decision Tree ===
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       113
           1       1.00      1.00      1.00       124
           2       1.00      1.00      1.00       114
           3       1.00      1.00      1.00       152

    accuracy                           1.00       503
   macro avg       1.00      1.00      1.00       503
weighted avg       1.00      1.00      1.00       503



Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [25]:

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='f1_macro')
grid_search.fit(X_train, y_train)
best_rf = grid_search.best_estimator_
y_pred_best_rf = best_rf.predict(X_test)


In [27]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


dt = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_dt = grid_search.best_estimator_
y_pred_best_dt = best_dt.predict(X_test)

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [26]:
print("=== Random Forest ===")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_best_rf))

=== Random Forest ===
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       113
           1       1.00      1.00      1.00       124
           2       1.00      1.00      1.00       114
           3       1.00      1.00      1.00       152

    accuracy                           1.00       503
   macro avg       1.00      1.00      1.00       503
weighted avg       1.00      1.00      1.00       503



In [28]:
# Evaluasi Model
print("=== Optimized Decision Tree ===")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Classification Report:\n", classification_report(y_test, y_pred_best_dt))

=== Optimized Decision Tree ===
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       113
           1       1.00      1.00      1.00       124
           2       1.00      1.00      1.00       114
           3       1.00      1.00      1.00       152

    accuracy                           1.00       503
   macro avg       1.00      1.00      1.00       503
weighted avg       1.00      1.00      1.00       503



Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.


**Perbandingan hasil**
1. Perbandingan hasil evaluasi sebelum dan setelah tuning
Hasil evaluasi sebelum dan setelah tuning menggunakan gridsearch tidak mengalami perubahan. Model tetap memiliki skor sempurna (1.0) di semua metrik.

**Kelemahan Model**
2. Model yang digunakan kemungkinan besar mengalami overfitting terhadap data latih, ditandai dengan akurasi 100% serta precision, recall, dan f1-score yang sempurna pada semua kelas. Hasil ini menunjukkan bahwa model telah menghafal pola dalam data tanpa benar-benar belajar untuk mengenali pola baru. Overfitting ini bisa disebabkan oleh kompleksitas model yang terlalu tinggi atau adanya kebocoran data antara data latih dan data uji. Selain itu, performa sempurna ini juga dapat mengindikasikan bahwa dataset yang digunakan terlalu sederhana atau tidak memiliki variasi yang cukup.

**Rekomendasi tindakan lanjutan**
- Kumpulkan lebih banyak data untuk meningkatkan generalisasi agar data lebih variatif

- Uji model pada dataset yang benar-benar baru untuk mengevaluasi kemampuannya dalam mengenali pola yang belum pernah dilihat sebelumnya.

- Analisis distribusi data guna memastikan bahwa dataset memiliki variasi yang cukup dan tidak terlalu mudah dipelajari oleh model.

- Lakukan feature selection agar model tidak terlalu bergantung pada fitur tertentu.